In [639]:
import pandas as pd
import numpy as np
import eli5

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder 
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from eli5.sklearn import PermutationImportance
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [640]:
df = pd.read_csv('oecd_ai_modified.csv')
df

,English name,Country,Start date,Yearly budget range,Has funding from private sector ?,Is evaluated ?,AI Principle(s),Policy instrument type category,National AI Policies,AI Governance
0,SPACERESOURCES.LU,Luxembourg,2016,3,0,0,1,Governance,1,0
1,DIGITAL LUXEMBOURG,Luxembourg,2014,2,0,1,1,AI enablers and other incentives,1,0
2,DIGITAL TECH FUND,Luxembourg,2016,2,0,0,1,Financial support,1,0
3,GAMEINN,Poland,2016,4,0,0,0,Financial support,1,0
4,POLAND-TAIWAN SCIENTIFIC CO-OPERATION,Poland,2012,1,0,0,0,Financial support,1,0
...,...,...,...,...,...,...,...,...,...,...
844,INVESTMENTS FOR THE FUTURE PROGRAMME,France,2010,7,0,1,1,Financial support,1,0
845,RESEARCH PLATFORMS,Belgium - Brussels Capital,2010,2,0,0,0,Financial support,1,0
846,INNOVATION FUND DENMARK,Denmark,2014,6,0,1,1,Financial support,1,0
847,ATHENA MAGAZINE,Belgium - Wallonia,1984,1,0,0,0,AI enablers and other incentives,1,0


In [641]:
df['Country'] = df['Country'].replace('Belgium - Wallonia', 'Belgium')
df['Country'] = df['Country'].replace('Belgium - Flanders', 'Belgium')
df['Country'] = df['Country'].replace('Belgium - Federal government', 'Belgium')
df['Country'] = df['Country'].replace('Belgium - Brussels Capital', 'Belgium')
df['Country'].value_counts()

Country
United States     74
European Union    59
United Kingdom    56
France            34
Australia         32
                  ..
Mauritius          2
Nigeria            2
Armenia            2
Croatia            1
Ukraine            1
Name: count, Length: 70, dtype: int64

In [642]:
a = pd.DataFrame(df['Country'].value_counts())
a.head(40)

,count
Country,
United States,74
European Union,59
United Kingdom,56
France,34
Australia,32
Türkiye,31
Germany,31
Colombia,28
Singapore,24


In [643]:
europe = ['Ukraine', 'Croatia', 'Armenia', 'Slovak Republic', 'Cyprus', 'Bulgaria', 'Romania', 'Greece', 'Lithuania', 'Iceland', 'Poland', 'Latvia','Malta',
          'Estonia', 'Luxembourg', 'Slovenia', 'Switzerland', 'Czech Republic', 'Ireland', 'Netherlands', 'Portugal', 'Russian Federation', 'Denmark',
          'Italy', 'Austria', 'European Union', 'United Kingdom', 'France', 'Germany', 'Belgium', 'Serbia', 'Norway', 'Spain','Hungary', 'Sweden', 'Finland']
africa = ['Nigeria', 'Mauritius', 'South Africa', 'Uganda', 'Morocco', 'Kenya', 'Egypt', 'Costa Rica', 'Tunisia', 'Rwanda']
north_america = ['United States', 'Canada']
south_america = ['Uruguay', 'Mexico', 'Peru', 'Brazil', 'Chile', 'Argentina', 'Colombia']
east_asia = ['China', 'Japan', 'Korea']
other_asia = ['Uzbekistan', 'Thailand', 'Kazakhstan', 'Viet Nam', 'Singapore']
middle_east = ['Saudi Arabia', 'United Arab Emirates', 'Israel', 'Türkiye', 'India']
oceania = ['New Zealand', 'Australia']

for i in europe:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'Europe'
for i in africa:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'Africa'
for i in north_america:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'North_America'
for i in south_america:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'South_America'
for i in east_asia:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'East_Asia'
for i in other_asia:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'Asia'
for i in middle_east:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'Middle_East'
for i in oceania:
    index_list = df[df['Country']==i].index.to_list()
    df.loc[index_list, 'Continent'] = 'Oceania'

In [709]:
#대시보드용 데이터
df.to_csv('data.csv', index=False)
df

,Country,Start date,Yearly budget range,Has funding from private sector ?,Is evaluated ?,AI Principle(s),Continent
0,Luxembourg,2016,3,0,0,1,Europe
1,Luxembourg,2014,2,0,1,1,Europe
2,Luxembourg,2016,2,0,0,1,Europe
3,Poland,2016,4,0,0,0,Europe
4,Poland,2012,1,0,0,0,Europe
...,...,...,...,...,...,...,...
844,France,2010,7,0,1,1,Europe
845,Belgium,2010,2,0,0,0,Europe
846,Denmark,2014,6,0,1,1,Europe
847,Belgium,1984,1,0,0,0,Europe


In [645]:
#ML용 데이터
df1 = df
df1.drop(['English name', 'Policy instrument type category', 'National AI Policies', 'AI Governance'], axis=1, inplace=True)
df1=df1.reset_index(drop=True)
sd = df1.copy()
df1

,Country,Start date,Yearly budget range,Has funding from private sector ?,Is evaluated ?,AI Principle(s),Continent
0,Luxembourg,2016,3,0,0,1,Europe
1,Luxembourg,2014,2,0,1,1,Europe
2,Luxembourg,2016,2,0,0,1,Europe
3,Poland,2016,4,0,0,0,Europe
4,Poland,2012,1,0,0,0,Europe
...,...,...,...,...,...,...,...
844,France,2010,7,0,1,1,Europe
845,Belgium,2010,2,0,0,0,Europe
846,Denmark,2014,6,0,1,1,Europe
847,Belgium,1984,1,0,0,0,Europe


In [646]:
df1.drop('Start date', axis=1, inplace=True)
df1

,Country,Yearly budget range,Has funding from private sector ?,Is evaluated ?,AI Principle(s),Continent
0,Luxembourg,3,0,0,1,Europe
1,Luxembourg,2,0,1,1,Europe
2,Luxembourg,2,0,0,1,Europe
3,Poland,4,0,0,0,Europe
4,Poland,1,0,0,0,Europe
...,...,...,...,...,...,...
844,France,7,0,1,1,Europe
845,Belgium,2,0,0,0,Europe
846,Denmark,6,0,1,1,Europe
847,Belgium,1,0,0,0,Europe


In [647]:
# 타겟 : 대륙

target = df1['Continent']
feature = df1.drop(['Continent'], axis=1)

In [648]:
# 훈련/평가 데이터를 분리하세요.
# 적절한 비율로 훈련/평가 데이터를 나누세요.
# random_state로 시드를 고정하세요.
# 타겟의 분포를 고려하여 데이터를 분리하세요.

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.25, stratify=target, random_state=42)

In [649]:
pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(max_depth = 11, n_estimators=30, random_state=42, oob_score=True, n_jobs=-1),
)
# cross_val_score() 함수를 사용해서 검증 성능을 확인
print('CV f1 score : ', np.mean(cross_val_score(pipe_rf, X_train, y_train, cv=3, scoring='f1_macro')))

pipe_rf.fit(X_train, y_train)
print("훈련 정확도: ", pipe_rf.score(X_train, y_train))
print("테스트 정확도: ", pipe_rf.score(X_test, y_test))

CV f1 score :  0.8169022597922914
훈련 정확도:  0.9355345911949685
테스트 정확도:  0.8262910798122066


In [650]:
# 타겟 : 대륙

target = df1['Continent']
feature = df1.drop(['Continent', 'Has funding from private sector ?', 'Is evaluated ?','Country'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.25, stratify=target, random_state=42)

In [651]:
feature

,Yearly budget range,AI Principle(s)
0,3,1
1,2,1
2,2,1
3,4,0
4,1,0
...,...,...
844,7,1
845,2,0
846,6,1
847,1,0


In [652]:
pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(max_depth = 12, n_estimators=100, random_state=42, oob_score=True, n_jobs=-1),
)
# cross_val_score() 함수를 사용해서 검증 성능을 확인
print('CV f1 score : ', np.mean(cross_val_score(pipe_rf, X_train, y_train, cv=3, scoring='f1_macro')))

pipe_rf.fit(X_train, y_train)
print("훈련 정확도: ", pipe_rf.score(X_train, y_train))
print("테스트 정확도: ", pipe_rf.score(X_test, y_test))

CV f1 score :  0.08306790824843359
훈련 정확도:  0.5031446540880503
테스트 정확도:  0.5070422535211268


In [653]:
#타겟 : 연간 예산
target1 = df1['Yearly budget range']
feature2 = df1.drop(['Yearly budget range', 'Country', 'Has funding from private sector ?', 'Is evaluated ?',], axis=1)
X_train, X_test, y_train, y_test = train_test_split(feature2, target1, test_size=0.25, stratify=target, random_state=42)

In [654]:
feature2

,AI Principle(s),Continent
0,1,Europe
1,1,Europe
2,1,Europe
3,0,Europe
4,0,Europe
...,...,...
844,1,Europe
845,0,Europe
846,1,Europe
847,0,Europe


In [655]:
pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(max_depth = 10, n_estimators=26, random_state=42, oob_score=True, n_jobs=-1),
)
# cross_val_score() 함수를 사용해서 검증 성능을 확인
print('CV f1 score : ', np.mean(cross_val_score(pipe_rf, X_train, y_train, cv=3, scoring='f1_macro')))

pipe_rf.fit(X_train, y_train)
print("훈련 정확도: ", pipe_rf.score(X_train, y_train))
print("테스트 정확도: ", pipe_rf.score(X_test, y_test))

CV f1 score :  0.10126678358799486
훈련 정확도:  0.6084905660377359
테스트 정확도:  0.647887323943662


In [656]:
#타겟 : 국가
target1 = df1['Country']
feature2 = df1.drop(['Country', 'Continent'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(feature2, target1, test_size=0.25, stratify=target, random_state=42)

In [657]:
pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(max_depth = 14, n_estimators=26, random_state=42, oob_score=True, n_jobs=-1),
)
# cross_val_score() 함수를 사용해서 검증 성능을 확인
print('CV f1 score : ', np.mean(cross_val_score(pipe_rf, X_train, y_train, cv=3, scoring='f1_macro')))

pipe_rf.fit(X_train, y_train)
print("훈련 정확도: ", pipe_rf.score(X_train, y_train))
print("테스트 정확도: ", pipe_rf.score(X_test, y_test))

/Users/nayeon/miniforge3/envs/sc4/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


CV f1 score :  0.03203082244373294
훈련 정확도:  0.22169811320754718
테스트 정확도:  0.20187793427230047


여기서부터는 새로운 데이터셋 조합------------------------

In [658]:
df2 = df1
df2

,Country,Yearly budget range,Has funding from private sector ?,Is evaluated ?,AI Principle(s),Continent
0,Luxembourg,3,0,0,1,Europe
1,Luxembourg,2,0,1,1,Europe
2,Luxembourg,2,0,0,1,Europe
3,Poland,4,0,0,0,Europe
4,Poland,1,0,0,0,Europe
...,...,...,...,...,...,...
844,France,7,0,1,1,Europe
845,Belgium,2,0,0,0,Europe
846,Denmark,6,0,1,1,Europe
847,Belgium,1,0,0,0,Europe


In [659]:
a = pd.DataFrame(df1['Country'].value_counts())
a['Doc_num'] = a['count']
a.drop('count', axis=1, inplace=True)
a = a.reset_index()
a

,Country,Doc_num
0,United States,74
1,European Union,59
2,United Kingdom,56
3,France,34
4,Australia,32
...,...,...
65,Mauritius,2
66,Nigeria,2
67,Armenia,2
68,Croatia,1


In [660]:
europe = ['Ukraine', 'Croatia', 'Armenia', 'Slovak Republic', 'Cyprus', 'Bulgaria', 'Romania', 'Greece', 'Lithuania', 'Iceland', 'Poland', 'Latvia','Malta',
          'Estonia', 'Luxembourg', 'Slovenia', 'Switzerland', 'Czech Republic', 'Ireland', 'Netherlands', 'Portugal', 'Russian Federation', 'Denmark',
          'Italy', 'Austria', 'European Union', 'United Kingdom', 'France', 'Germany', 'Belgium', 'Serbia', 'Norway', 'Spain','Hungary', 'Sweden', 'Finland']
africa = ['Nigeria', 'Mauritius', 'South Africa', 'Uganda', 'Morocco', 'Kenya', 'Egypt', 'Costa Rica', 'Tunisia', 'Rwanda']
north_america = ['United States', 'Canada']
south_america = ['Uruguay', 'Mexico', 'Peru', 'Brazil', 'Chile', 'Argentina', 'Colombia']
east_asia = ['China', 'Japan', 'Korea']
other_asia = ['Uzbekistan', 'Thailand', 'Kazakhstan', 'Viet Nam', 'Singapore']
middle_east = ['Saudi Arabia', 'United Arab Emirates', 'Israel', 'Türkiye', 'India']
oceania = ['New Zealand', 'Australia']

for i in europe:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'Europe'
for i in africa:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'Africa'
for i in north_america:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'North_America'
for i in south_america:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'South_America'
for i in east_asia:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'East_Asia'
for i in other_asia:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'Asia'
for i in middle_east:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'Middle_East'
for i in oceania:
    index_list = a[a['Country']==i].index.to_list()
    a.loc[index_list, 'Continent'] = 'Oceania'

a = a.sort_values(by='Country')
a = a.reset_index(drop=True)
a

,Country,Doc_num,Continent
0,Argentina,9,South_America
1,Armenia,2,Europe
2,Australia,32,Oceania
3,Austria,9,Europe
4,Belgium,22,Europe
...,...,...,...
65,United Kingdom,56,Europe
66,United States,74,North_America
67,Uruguay,3,South_America
68,Uzbekistan,3,Asia


In [661]:
df1.groupby('Country')['Yearly budget range'].sum()

Country
Argentina         10
Armenia            0
Australia         55
Austria            3
Belgium           44
                  ..
United Kingdom    64
United States     32
Uruguay            3
Uzbekistan         0
Viet Nam           2
Name: Yearly budget range, Length: 70, dtype: int64

In [662]:
date = pd.DataFrame(sd.groupby('Country')['Start date'].min())
date = date.reset_index()
date
a['Start date'] = date['Start date']
a

,Country,Doc_num,Continent,Start date
0,Argentina,9,South_America,2001
1,Armenia,2,Europe,2002
2,Australia,32,Oceania,1991
3,Austria,9,Europe,1999
4,Belgium,22,Europe,1984
...,...,...,...,...
65,United Kingdom,56,Europe,2000
66,United States,74,North_America,1999
67,Uruguay,3,South_America,2019
68,Uzbekistan,3,Asia,2020


In [663]:
coun = pd.DataFrame(a['Country'].sort_values())
coun.head(40)

,Country
0,Argentina
1,Armenia
2,Australia
3,Austria
4,Belgium
5,Brazil
6,Bulgaria
7,Canada
8,Chile
9,China


In [664]:
#gdp, 2020 월드뱅크 기준. EU 빼고 전부 다. 빌리언 달러 기준.
d = pd.DataFrame()
d['GDP'] = ['Argentina:$449.7 billion (2020)',
'Armenia: $13.7 billion (2020)',
'Australia: $1488 trillion (2020)',
'Austria: $455.6 billion (2020)',
'Belgium: $540.2 billion (2020)',
'Brazil: $1449 trillion (2020)',
'Bulgaria: $69.2 billion (2020)',
'Canada: $1646 trillion (2020)',
'Chile: $247.9 billion (2020)',
'China: $16640 trillion (2020)',
'Colombia: $303.5 billion (2020)',
'Costa Rica: $64.6 billion (2020)',
'Croatia: $63.2 billion (2020)',
'Cyprus: $24.8 billion (2020)',
'Czech Republic: $256.1 billion (2020)',
'Denmark: $324.4 billion (2020)',
'Egypt: $362.8 billion (2020)',
'Estonia: $31.2 billion (2020)',
'EU:?',
'Finland: $284.5 billion (2020)',
'France: $2715 trillion (2020)',
'Germany: $3861 trillion (2020)',
'Greece: $204.9 billion (2020)',
'Hungary: $170.6 billion (2020)',
'Iceland: $25.7 billion (2020)',
'India: $2700 trillion (2020)',
'Indonesia: $1060 trillion (2020)',
'Ireland: $377.6 billion (2020)',
'Israel: $395.1 billion (2020)',
'Italy: $2003 trillion (2020)',
'Japan: $5378 trillion (2020)',
'Kazakhstan: $184.4 billion (2020)',
'Kenya: $95.5 billion (2020)',
'Korea: $1645 trillion (2020)',
'Latvia: $34.9 billion (2020)',
'Lithuania: $58.5 billion (2020)',
'Luxembourg: $72.5 billion (2020)',
'Malta: $15.7 billion (2020)',
'Mauritius: $14.3 billion (2020)',
'Mexico: $1046 trillion (2020)',
'Morocco: $122.9 billion (2020)',
'Netherlands: $912.8 billion (2020)',
'New Zealand: $207.4 billion (2020)',
'Nigeria: $448.1 billion (2020)',
'Norway: $398.8 billion (2020)',
'Peru: $227.1 billion (2020)',
'Poland: $592.1 billion (2020)',
'Portugal: $247.9 billion (2020)',
'Romania: $261.8 billion (2020)',
'Rwanda: $10.5 billion (2020)',
'Saudi Arabia: $1685 trillion (2020)',
'Serbia: $53.6 billion (2020)',
'Singapore: $337.9 billion (2020)',
'Slovakia: $105.9 billion (2020)',
'Slovenia: $55.8 billion (2020)',
'South Africa: $283.4 billion (2020)',
'Spain: $1416 trillion (2020)',
'Sweden: $527.6 billion (2020)',
'Switzerland: $710.9 billion (2020)',
'Thailand: $515.9 billion (2020)',
'Tunisia: $40.8 billion (2020)',
'Turkey: $717.9 billion (2020)',
'Uganda: $39.8 billion (2020)',
'Ukraine: $153.6 billion (2020)',
'United Arab Emirates: $421.1 billion (2020)',
'United Kingdom: $2631 trillion (2020)',
'United States: $22675 trillion (2020)',
'Uruguay: $58.2 billion (2020)',
'Uzbekistan: $64.2 billion (2020)',
'Vietnam: $340.6 billion (2020)']
a['GDP'] = d['GDP']
a

,Country,Doc_num,Continent,Start date,GDP
0,Argentina,9,South_America,2001,Argentina:$449.7 billion (2020)
1,Armenia,2,Europe,2002,Armenia: $13.7 billion (2020)
2,Australia,32,Oceania,1991,Australia: $1488 trillion (2020)
3,Austria,9,Europe,1999,Austria: $455.6 billion (2020)
4,Belgium,22,Europe,1984,Belgium: $540.2 billion (2020)
...,...,...,...,...,...
65,United Kingdom,56,Europe,2000,United Kingdom: $2631 trillion (2020)
66,United States,74,North_America,1999,United States: $22675 trillion (2020)
67,Uruguay,3,South_America,2019,Uruguay: $58.2 billion (2020)
68,Uzbekistan,3,Asia,2020,Uzbekistan: $64.2 billion (2020)


In [665]:
import re
a['GDP'] = a['GDP'].apply(lambda x: re.sub(r'^.*\$', '', str(x)))
a['GDP'] = a['GDP'].str.split().str[0]
a

,Country,Doc_num,Continent,Start date,GDP
0,Argentina,9,South_America,2001,449.7
1,Armenia,2,Europe,2002,13.7
2,Australia,32,Oceania,1991,1488
3,Austria,9,Europe,1999,455.6
4,Belgium,22,Europe,1984,540.2
...,...,...,...,...,...
65,United Kingdom,56,Europe,2000,2631
66,United States,74,North_America,1999,22675
67,Uruguay,3,South_America,2019,58.2
68,Uzbekistan,3,Asia,2020,64.2


In [666]:
#예산 배정 액수. 이 데이터에서는 실제 금액이 아닌 양으로 해석해야할 듯.
#{"Don't know": 0,'Not applicable':0, 'Less than 1M':1, '1M-5M':2, '5M-20M':3, '20M-50M':4,'50M-100M':5, '100M-500M':6, 'More than 500M':7}

budget = pd.DataFrame(sd.groupby('Country')['Yearly budget range'].sum())
budget = budget.reset_index()
budget
a['Yearly budget'] = budget['Yearly budget range']
a

,Country,Doc_num,Continent,Start date,GDP,Yearly budget
0,Argentina,9,South_America,2001,449.7,10
1,Armenia,2,Europe,2002,13.7,0
2,Australia,32,Oceania,1991,1488,55
3,Austria,9,Europe,1999,455.6,3
4,Belgium,22,Europe,1984,540.2,44
...,...,...,...,...,...,...
65,United Kingdom,56,Europe,2000,2631,64
66,United States,74,North_America,1999,22675,32
67,Uruguay,3,South_America,2019,58.2,3
68,Uzbekistan,3,Asia,2020,64.2,0


In [710]:
a.to_csv('ml_data.csv', index=False)

In [667]:
#gdp, 예산 배정 액수, 문서 건수, 시작 연도, 대륙
#최종 모델!!!!!!!!!

In [707]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


target = a['Doc_num']
feature = a.drop('Doc_num', axis=1)

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(feature, target, test_size=0.25, random_state=42)

pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(max_depth = 40, n_estimators=500, min_samples_split=7, random_state=42),
)

pipe_rf.fit(X_train, y_train)

print("훈련 정확도: ", pipe_rf.score(X_train, y_train))
print("검증 정확도: ", pipe_rf.score(X_val, y_val))
print("최종 정확도: ", pipe_rf.score(X_test, y_test))

y_val_pred = pipe_rf.predict(X_val)
y_test_pred = pipe_rf.predict(X_test)

mse = mean_squared_error(y_val, y_val_pred)
print("검증 MSE:", mse)
mse = mean_squared_error(y_test, y_test_pred)
print("최종 MSE:", mse)

r2 = r2_score(y_val, y_val_pred)
print("검증 R-squared score:", r2)
r2 = r2_score(y_test, y_test_pred)
print("최종 R-squared score:", r2)


훈련 정확도:  0.7163460487771857
검증 정확도:  0.6376347704032721
최종 정확도:  0.6376347704032721
검증 MSE: 68.24545157405043
최종 MSE: 68.24545157405043
검증 R-squared score: 0.6376347704032721
최종 R-squared score: 0.6376347704032721


In [711]:
#모델 피클링
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(pipe_rf, pickle_file)